In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

/home/bis/miniconda3/envs/TF1.14/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/bis/miniconda3/envs/TF1.14/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/bis/miniconda3/envs/TF1.14/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/bis/miniconda3/envs/TF1.14/lib/python3.6/site-

In [2]:
import model.lstm_tf as lstm
import model.fragmentation_config as fconfig


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



# MS2 - Load Model

In [3]:
# other models are also included in the './tf-models' folder.
model_folder = './tf-models/model-180921-modloss/'
model = 'pretrain-180921-modloss.ckpt'

ion_types = ['b{}', 'y{}']
mod_config = fconfig.HCD_CommonMod_Config()
mod_config.SetFixMod(['Carbamidomethyl[C]'])
mod_config.varmod = ["Oxidation[M]"]
mod_config.SetIonTypes(ion_types)
mod_config.time_step = 100
mod_config.min_var_mod_num = 0
mod_config.max_var_mod_num = 2

pdeep = lstm.IonLSTM(mod_config)

In [4]:
pdeep.LoadModel(model_file = model_folder + model)




Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./tf-models/model-180921-modloss/pretrain-180921-modloss.ckpt



In [5]:
model_file = model_folder + model
saver = tf.train.import_meta_graph(model_file+".meta")

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
config.intra_op_parallelism_threads = 8
sess = tf.Session(config=config)

init_op = tf.global_variables_initializer()
sess.run(init_op)

saver.restore(sess, model_file)
graph = tf.get_default_graph()

In [9]:
total_parameters = 0
for variable in tf.trainable_variables():
    # shape is an array of tf.Dimension
    shape = variable.get_shape()
    print(shape)
    print(len(shape))
    variable_parameters = 1
    for dim in shape:
        print(dim)
        variable_parameters *= dim.value
    print(variable_parameters)
    total_parameters += variable_parameters
print(total_parameters)

(9, 3)
2
9
3
27
(355, 1024)
2
355
1024
363520
(1024,)
1
1024
1024
(355, 1024)
2
355
1024
363520
(1024,)
1
1024
1024
(772, 1024)
2
772
1024
790528
(1024,)
1
1024
1024
(772, 1024)
2
772
1024
790528
(1024,)
1
1024
1024
(524, 32)
2
524
32
16768
(32,)
1
32
32
(524, 32)
2
524
32
16768
(32,)
1
32
32
(9, 3)
2
9
3
27
(355, 1024)
2
355
1024
363520
(1024,)
1
1024
1024
(355, 1024)
2
355
1024
363520
(1024,)
1
1024
1024
(772, 1024)
2
772
1024
790528
(1024,)
1
1024
1024
(772, 1024)
2
772
1024
790528
(1024,)
1
1024
1024
(524, 32)
2
524
32
16768
(32,)
1
32
32
(524, 32)
2
524
32
16768
(32,)
1
32
32
4691638


In [11]:
np.sum([np.prod(v.get_shape().as_list()) for v in tf.trainable_variables()])

4691638

# package

In [ ]:
Count the number of parameters in ckpt file(tf-models/model-180921-modloss/pretrain-180921-modloss.ckpt)
output_scope/backbone_rnn/fw/lstm_cell/kernel: [524, 32] => 16768
output_scope/backbone_rnn/fw/lstm_cell/bias/Adam: [32] => 32
output_scope/backbone_rnn/bw/lstm_cell/kernel/Adam_1: [524, 32] => 16768
output_scope/backbone_rnn/bw/lstm_cell/kernel/Adam: [524, 32] => 16768
output_scope/backbone_rnn/bw/lstm_cell/kernel: [524, 32] => 16768
output_scope/backbone_rnn/fw/lstm_cell/bias: [32] => 32
output_scope/backbone_rnn/bw/lstm_cell/bias/Adam_1: [32] => 32
output_scope/backbone_rnn/fw/lstm_cell/bias/Adam_1: [32] => 32
output_scope/backbone_rnn/bw/lstm_cell/bias/Adam: [32] => 32
instrument_nce/scan_w/Adam: [9, 3] => 27
output_scope/backbone_rnn/fw/lstm_cell/kernel/Adam_1: [524, 32] => 16768
beta2_power: [] => 1
beta1_power: [] => 1
BiLSTM_1/fw/lstm_cell/kernel/Adam_1: [772, 1024] => 790528
BiLSTM_0/fw/lstm_cell/kernel: [355, 1024] => 363520
BiLSTM_0/fw/lstm_cell/bias/Adam_1: [1024] => 1024
instrument_nce/scan_w: [9, 3] => 27
BiLSTM_0/fw/lstm_cell/bias: [1024] => 1024
BiLSTM_0/fw/lstm_cell/kernel/Adam: [355, 1024] => 363520
output_scope/backbone_rnn/fw/lstm_cell/kernel/Adam: [524, 32] => 16768
BiLSTM_0/fw/lstm_cell/bias/Adam: [1024] => 1024
BiLSTM_1/fw/lstm_cell/kernel: [772, 1024] => 790528
BiLSTM_1/bw/lstm_cell/bias/Adam: [1024] => 1024
BiLSTM_0/bw/lstm_cell/kernel/Adam_1: [355, 1024] => 363520
output_scope/backbone_rnn/bw/lstm_cell/bias: [32] => 32
BiLSTM_0/bw/lstm_cell/bias/Adam: [1024] => 1024
BiLSTM_1/bw/lstm_cell/kernel: [772, 1024] => 790528
BiLSTM_0/bw/lstm_cell/bias/Adam_1: [1024] => 1024
BiLSTM_0/bw/lstm_cell/kernel/Adam: [355, 1024] => 363520
BiLSTM_0/fw/lstm_cell/kernel/Adam_1: [355, 1024] => 363520
BiLSTM_1/bw/lstm_cell/bias: [1024] => 1024
BiLSTM_1/bw/lstm_cell/bias/Adam_1: [1024] => 1024
BiLSTM_1/fw/lstm_cell/kernel/Adam: [772, 1024] => 790528
BiLSTM_0/bw/lstm_cell/kernel: [355, 1024] => 363520
BiLSTM_1/bw/lstm_cell/kernel/Adam: [772, 1024] => 790528
BiLSTM_1/bw/lstm_cell/kernel/Adam_1: [772, 1024] => 790528
BiLSTM_0/bw/lstm_cell/bias: [1024] => 1024
BiLSTM_1/fw/lstm_cell/bias: [1024] => 1024
instrument_nce/scan_w/Adam_1: [9, 3] => 27
BiLSTM_1/fw/lstm_cell/bias/Adam_1: [1024] => 1024
BiLSTM_1/fw/lstm_cell/bias/Adam: [1024] => 1024
Total Param Count: 7037459


# not pretrained model

In [12]:
ion_types = ['b{}', 'y{}']
mod_config = fconfig.HCD_CommonMod_Config()
mod_config.SetFixMod(['Carbamidomethyl[C]'])
mod_config.varmod = ["Oxidation[M]"]
mod_config.SetIonTypes(ion_types)
mod_config.time_step = 100
mod_config.min_var_mod_num = 0
mod_config.max_var_mod_num = 2

pdeep = lstm.IonLSTM(mod_config)

pdeep.learning_rate = 0.001
pdeep.layer_size = 256
pdeep.batch_size = 1024
pdeep.BuildModel(input_size = 98, output_size = mod_config.GetTFOutputSize(), nlayers = 2)

BuildModel ... 



Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [37]:
print(mod_config.GetTFOutputSize())

4


In [39]:
total_parameters = 0
for variable in tf.trainable_variables():
    # shape is an array of tf.Dimension
    shape = variable.get_shape()
    print(shape)
    print(len(shape))
    variable_parameters = 1
    for dim in shape:
#         print(dim)
        variable_parameters *= dim.value
    print(variable_parameters)
    total_parameters += variable_parameters
print(total_parameters)

(9, 3)
2
27
(355, 1024)
2
363520
(1024,)
1
1024
(355, 1024)
2
363520
(1024,)
1
1024
(772, 1024)
2
790528
(1024,)
1
1024
(772, 1024)
2
790528
(1024,)
1
1024
(520, 16)
2
8320
(16,)
1
16
(520, 16)
2
8320
(16,)
1
16
2328891


In [22]:
np.sum([np.prod(v.get_shape().as_list()) for v in tf.trainable_variables()])

2328891

# data

In [8]:
pt_g6 = pd.read_csv('data/ProteomeTools/01625b_GA6-TUM_first_pool_41_01_01-3xHCD-1h-R1.plabel', sep='\t')
pt_g7 = pd.read_csv('data/ProteomeTools/01625b_GA7-TUM_first_pool_120_01_01-3xHCD-1h-R1.plabel', sep='\t')

In [9]:
print(pt_g6.shape, pt_g7.shape)

(13004, 11) (9431, 11)


In [10]:
pt_g6.head(2)

,spec,peptide,modinfo,b,b-NH3,b-H2O,b-ModLoss,y,y-NH3,y-H2O,y-ModLoss
0,01625b_GA6-TUM_first_pool_41_01_01-3xHCD-1h-R1...,HLDLIEMLEAK,"7,Oxidation[M];","b1+1,325014.9;b2+1,3336692.3;b3+1,782299.1;b4+...","b2-NH3+1,99423.1;b6-NH3+1,24342.3;b8-NH3+1,263...","b3-H2O+1,569043.4;b5-H2O+1,38893.7;b6-H2O+1,79...","b7-ModLoss+1,289430.5;b7-ModLoss+2,105029.3;b8...","y10+1,644353.9;y9+1,1524192.6;y8+1,371562.3;y7...","y9-NH3+1,50124.9;y6-NH3+1,26595.2;y2-NH3+1,962...","y9-H2O+1,77871.4;y6-H2O+1,65700.5;y3-H2O+1,350...","y10-ModLoss+1,733900.8;y9-ModLoss+1,1696219.9;..."
1,01625b_GA6-TUM_first_pool_41_01_01-3xHCD-1h-R1...,HLDLIEMLEAK,"7,Oxidation[M];","b1+1,76364.2;b2+1,750020.6;b3+1,183478;b4+1,17...",NaN,"b3-H2O+1,128524.2;b6-H2O+1,27056;b7-H2O+1,3270...","b7-ModLoss+1,79250.6;b7-ModLoss+2,22529.3;b8-M...","y10+1,110337;y9+1,324225.4;y8+1,82257.2;y7+1,5...","y2-NH3+1,91712.6;y1-NH3+1,96104;","y9-H2O+1,21417.1;y6-H2O+1,18130;y3-H2O+1,76746...","y10-ModLoss+1,160679.6;y9-ModLoss+1,355289.2;y..."
